# Inference for classification pipeline

Inference is working on prepared image cutouts, not library images as a whole.

# Preprocessing

run once to load methods

In [1]:
import cv2
import numpy as np
from typing import Tuple, Union
import math

# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal - blur
def blur(image):
    return cv2.medianBlur(image,5)
 
# thresholding
def thresholding(image):
    return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 15)

# dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
# erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

# opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# closing - dilation followed by erosion
def closing(image):
    kernel = np.ones((1,1),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

def rotate(image: np.ndarray, angle: float, background: Union[int, Tuple[int, int, int]]) -> np.ndarray:
    old_width, old_height = image.shape[:2]
    angle_radian = math.radians(angle)
    width = abs(np.sin(angle_radian) * old_height) + abs(np.cos(angle_radian) * old_width)
    height = abs(np.sin(angle_radian) * old_width) + abs(np.cos(angle_radian) * old_height)

    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    rot_mat[1, 2] += (width - old_width) / 2
    rot_mat[0, 2] += (height - old_height) / 2
    return cv2.warpAffine(image, rot_mat, (int(round(height)), int(round(width))), borderValue=background, borderMode=cv2.BORDER_CONSTANT)

# Setup

loading every reference value

In [2]:
hu_baseline = []

with open('path-to-hu-baseline-arial-normal') as h: # path to prepared file containing Hu baseline numbers for normal Arial font
  lines = h.readlines()

for x in lines:
  hu_baseline.append(x.strip())

with open('path-to-hu-baseline-arial-bold') as h: # path to prepared file containing Hu baseline numbers for bold Arial font
  lines = h.readlines()

for x in lines:
  hu_baseline.append(x.strip())


In [ ]:
!pip install deskew

# Inference on folder

## without deskewing

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from math import copysign, log10
import functools
import os


directory = 'path-to-inference-folder'

i, sma, smb = 0, 0, 0
eps = 1.e-5
mmm = 0
result = 0;
anyA, anyB = False, False
results = {}
radius = 8
output = []

for filename in sorted(os.listdir(directory)):
  if filename.endswith('.jpg') or filename.endswith('.png'):
    prediction = ''
    image = cv2.imread(os.path.join(directory,filename))
    
    gray = get_grayscale(image)

    b = blur(gray)
    t = thresholding(b)
    o = opening(t)
    cl = closing(o)
    thresh = cv2.bitwise_not(cl)

    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    coord = []
    for contour in contours:  
          [x,y,w,h] = cv2.boundingRect(contour)
          if h>6 and w>60:   
              continue   
          if h<5 or w<3:   
              continue  
          coord.append((x,y,w,h))

    def compare(rect1, rect2):
        if abs(rect1[1] - rect2[1]) > 10:
            return rect1[1] - rect2[1]
        else:
            return rect1[0] - rect2[0]

    coord = sorted(coord, key=functools.cmp_to_key(compare))

    buchstaben = []
    hu = []
    count = 0
    for cor in coord:
      [x,y,w,h] = cor
      t = thresh[y:y+h,x:x+w]
      img = cv2.copyMakeBorder(t, 6, 5, 5, 5, cv2.BORDER_CONSTANT, value=0)
      img = cv2.resize(img, (32, 32))

      x = cv2.moments(img)
      x = cv2.HuMoments(x)
      # Log scale hu moments 
      for i in range(0,7):
        if x[i] > 0:
          x[i] = -1* copysign(1.0, x[i]) * log10(abs(x[i]))

      hu.append(x)

    for x in range(len(hu)):
      ma = hu[x]
      for c in range(len(hu_baseline)):
        mb = hu_baseline[c][4:]
        mb = mb.split(' ')
        mb = np.asarray(mb, dtype=np.float64)
        key = hu_baseline[c][0:3]
        result = 0
        for i in range(7):

          ama = abs(ma[i])
          amb = abs(mb[i])

          if (ama > 0):
            anyA = True
          if (amb > 0):
            anyB = True

          if (ma[i] > 0):
            sma = 1
          elif (ma[i] < 0):
            sma = -1
          else:
            sma = 0
          if (mb[i] > 0):
            smb = 1
          elif (mb[i] < 0):
            smb = -1
          else:
            smb = 0
          if (ama > eps and amb > eps):

            ama = sma * log10(ama)
            amb = smb * log10(amb)
            result += abs(-ama + amb)

        if (anyA != anyB):
          result = sys.float_info.max

        results[key] = result

      prediction += min(results, key=results.get)[0]

    output.append(prediction)

## with deskewing

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from math import copysign, log10
import functools
import os
from deskew import determine_skew


directory = 'path-to-inference-folder'

i, sma, smb = 0, 0, 0
eps = 1.e-5
mmm = 0
result = 0;
anyA, anyB = False, False
results = {}
radius = 8
output = []

for filename in sorted(os.listdir(directory)):
  if filename.endswith('.jpg') or filename.endswith('.png'):
    prediction = ''
    image = cv2.imread(os.path.join(directory,filename))
    
    gray = get_grayscale(image)
    angle = determine_skew(gray)
    if angle is not None:
      if abs(angle)>45:
        angle = angle + 90
      rotated = rotate(gray, angle, (255, 255, 255))
    else:
      rotated=gray

    b = blur(rotated)
    t = thresholding(b)
    o = opening(t)
    cl = closing(o)
    thresh = cv2.bitwise_not(cl)

    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    coord = []
    for contour in contours:  
          [x,y,w,h] = cv2.boundingRect(contour)
          if h>6 and w>60:   
              continue   
          if h<5 or w<3:   
              continue  
          coord.append((x,y,w,h))

    def compare(rect1, rect2):
        if abs(rect1[1] - rect2[1]) > 10:
            return rect1[1] - rect2[1]
        else:
            return rect1[0] - rect2[0]

    coord = sorted(coord, key=functools.cmp_to_key(compare))

    buchstaben = []
    hu = []
    count = 0
    for cor in coord:
      [x,y,w,h] = cor
      t = thresh[y:y+h,x:x+w]
      img = cv2.copyMakeBorder(t, 6, 5, 5, 5, cv2.BORDER_CONSTANT, value=0)
      img = cv2.resize(img, (32, 32))

      x = cv2.moments(img)
      x = cv2.HuMoments(x)
      # Log scale hu moments 
      for i in range(0,7):
        if x[i] > 0:
          x[i] = -1* copysign(1.0, x[i]) * log10(abs(x[i]))

      hu.append(x)

    for x in range(len(hu)):
      ma = hu[x]
      for c in range(len(hu_baseline)):
        mb = hu_baseline[c][4:]
        mb = mb.split(' ')
        mb = np.asarray(mb, dtype=np.float64)
        key = hu_baseline[c][0:3]
        result = 0
        for i in range(7):

          ama = abs(ma[i] );
          amb = abs(mb[i] );

          if (ama > 0):
            anyA = True
          if (amb > 0):
            anyB = True

          if ( ma[i] > 0 ):
            sma = 1
          elif ( ma[i] < 0 ):
            sma = -1
          else:
            sma = 0
          if ( mb[i] > 0 ):
            smb = 1
          elif ( mb[i] < 0 ):
            smb = -1
          else:
            smb = 0
          if ( ama > eps and amb > eps ):

            ama = sma * log10( ama )
            amb = smb * log10( amb )
            result += abs( -ama + amb )

        if (anyA != anyB):
          result = sys.float_info.max

        results[key] = result

      prediction += min(results, key=results.get)[0]

    output.append(prediction)